# Part 3 - Themes Over Time and Party Affiliation

- What are the most common themes in inaugural addresses?  
- How has this changed over time?
- Does a certain party have more polarizing speech?

In [23]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter

In [2]:
inaugural = pd.read_csv('data/inaugural_address.csv').iloc[:, 1:]
#convert to dt
inaugural['date'] = pd.to_datetime(inaugural['date'])
inaugural.head()

,president_name,president_number,date,text
0,George Washington,1,1789-04-30 00:00:00+00:00,\nFellow-Citizens of the Senate and of the Hou...
1,George Washington,1,1793-03-04 00:00:00+00:00,\nFellow Citizens:\nI AM again called upon by ...
2,John Adams,2,1797-03-04 00:00:00+00:00,"\nWHEN it was first perceived, in early times,..."
3,Thomas Jefferson,3,1801-03-04 00:00:00+00:00,\nFriends and Fellow-Citizens:\nCALLED upon to...
4,Thomas Jefferson,3,1805-03-04 00:00:00+00:00,"\nPROCEEDING, fellow-citizens, to that qualifi..."


### What are the most common themes in inaugural addresses?

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models
from spacy import displacy


In [ ]:
# If did not get env from Makefile, run:
!python -m spacy download en_core_web_sm

In [4]:
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text): 
    doc = nlp(text) 
    return [
        token.lemma_.lower() for token in doc 
        if not (token.is_stop or token.is_punct or token.is_space) 
        and len(token.lemma_) > 3
    ]

In [ ]:
processed_docs = inaugural["text"].apply(preprocess_text)

dictionary = Dictionary(processed_docs) 
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs] # approx 25 seconds

In [6]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

In [7]:
print("Inaugural Addresses LDA Themes:")
for i, topic in lda_model.print_topics(-1): 
    print(f"Theme: {i}")
    print(f"Words: {topic}")
    print()

Inaugural Addresses LDA Themes:
Theme: 0
Words: 0.005*"method" + 0.004*"establish" + 0.004*"race" + 0.004*"community" + 0.004*"rest" + 0.004*"commercial" + 0.004*"officer" + 0.004*"local" + 0.004*"relation" + 0.004*"election"

Theme: 1
Words: 0.010*"business" + 0.006*"federal" + 0.006*"increase" + 0.005*"ought" + 0.005*"legislation" + 0.004*"tariff" + 0.004*"revenue" + 0.004*"proper" + 0.004*"race" + 0.004*"trade"

Theme: 2
Words: 0.009*"thank" + 0.008*"today" + 0.007*"like" + 0.006*"task" + 0.006*"face" + 0.005*"child" + 0.005*"civilization" + 0.004*"industrial" + 0.004*"help" + 0.004*"wish"

Theme: 3
Words: 0.013*"today" + 0.012*"americans" + 0.009*"century" + 0.007*"democracy" + 0.007*"child" + 0.006*"generation" + 0.006*"earth" + 0.006*"promise" + 0.006*"challenge" + 0.006*"help"

Theme: 4
Words: 0.005*"opinion" + 0.005*"object" + 0.004*"general" + 0.004*"exist" + 0.004*"revenue" + 0.004*"limit" + 0.004*"regard" + 0.003*"circumstance" + 0.003*"effect" + 0.003*"experience"



In [8]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.092054  0.057403       1        1  42.045476
3      0.150504  0.030558       2        1  21.736841
1     -0.064091 -0.076400       3        1  16.356031
0     -0.072022  0.022137       4        1   9.949982
2      0.077663 -0.033698       5        1   9.911670, topic_info=           Term        Freq       Total Category  logprob  loglift
1539      today  104.000000  104.000000  Default  30.0000  30.0000
1584  americans   82.000000   82.000000  Default  29.0000  29.0000
621       thank   47.000000   47.000000  Default  28.0000  28.0000
1254    century   71.000000   71.000000  Default  27.0000  27.0000
1319   business   64.000000   64.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1133      trade    8.892214   49.271679   Topic5  -5.7208   0.5993
387     mankind    8.104704   36.788487   Topic5  -5.8135   0.7987
989       money    8.042290   38.610689   Topic5  -5.8212   0.7426
819        fail    8.192733   47.075103   Topic5  -5.8027   0.5630
1340      dream    7.840187   40.324197   Topic5  -5.8467   0.6738

[357 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1042      1  0.158154     accept
1042      2  0.289949     accept
1042      3  0.210872     accept
1042      4  0.263590     accept
1042      5  0.079077     accept
...     ...       ...        ...
799       2  0.608201       word
799       3  0.045052       word
799       5  0.157682       word
1609      2  0.557469  yesterday
1609      5  0.371646  yesterday

[1006 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 1, 3])

### How have the major themes changed over time?

In [9]:
inaugural_1700s = inaugural[inaugural['date'].dt.year < 1800]
inaugural_1800s = inaugural[(1800 <= inaugural['date'].dt.year) & (inaugural['date'].dt.year < 1900)]
inaugural_1900s = inaugural[(1900 <= inaugural['date'].dt.year) & (inaugural['date'].dt.year < 2000)]
inaugural_2000s = inaugural[2000 < inaugural['date'].dt.year]

In [10]:
processed_docs_1700s = inaugural_1700s["text"].apply(preprocess_text)
processed_docs_1800s = inaugural_1800s["text"].apply(preprocess_text)
processed_docs_1900s = inaugural_1900s["text"].apply(preprocess_text)
processed_docs_2000s = inaugural_2000s["text"].apply(preprocess_text)

# use same dictionary for all:
dictionary = Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus_1700s = [dictionary.doc2bow(doc) for doc in processed_docs_1700s]
corpus_1800s = [dictionary.doc2bow(doc) for doc in processed_docs_1800s]
corpus_1900s = [dictionary.doc2bow(doc) for doc in processed_docs_1900s]
corpus_2000s = [dictionary.doc2bow(doc) for doc in processed_docs_2000s]

lda_model_1700s = LdaModel(corpus=corpus_1700s, id2word=dictionary, num_topics=5, random_state=42, passes=10)
lda_model_1800s = LdaModel(corpus=corpus_1800s, id2word=dictionary, num_topics=5, random_state=42, passes=10)
lda_model_1900s = LdaModel(corpus=corpus_1900s, id2word=dictionary, num_topics=5, random_state=42, passes=10)
lda_model_2000s = LdaModel(corpus=corpus_2000s, id2word=dictionary, num_topics=5, random_state=42, passes=10)

In [11]:
print("Inaugural Addresses 18th Century LDA Themes:")
for i, topic in lda_model_1700s.print_topics(-1): 
    print(f"Theme: {i}")
    print(f"Words: {topic}")
    print()

Inaugural Addresses 18th Century LDA Themes:
Theme: 0
Words: 0.003*"voice" + 0.003*"execute" + 0.003*"official" + 0.003*"entertain" + 0.003*"instance" + 0.003*"constitutional" + 0.003*"function" + 0.003*"occasion" + 0.003*"endeavor" + 0.003*"presence"

Theme: 1
Words: 0.001*"knowledge" + 0.001*"general" + 0.001*"legislature" + 0.001*"choice" + 0.001*"establish" + 0.001*"resolution" + 0.001*"virtuous" + 0.001*"wish" + 0.001*"feel" + 0.001*"idea"

Theme: 2
Words: 0.001*"happiness" + 0.001*"ought" + 0.001*"establish" + 0.001*"blessing" + 0.001*"humble" + 0.001*"nature" + 0.001*"influence" + 0.001*"particular" + 0.001*"decide" + 0.001*"expedient"

Theme: 3
Words: 0.007*"ought" + 0.005*"happiness" + 0.005*"establish" + 0.005*"circumstance" + 0.005*"nature" + 0.004*"private" + 0.004*"influence" + 0.004*"blessing" + 0.004*"execute" + 0.004*"particular"

Theme: 4
Words: 0.008*"legislature" + 0.006*"virtuous" + 0.006*"general" + 0.006*"choice" + 0.006*"knowledge" + 0.004*"happiness" + 0.004*"co

In [12]:
print("Inaugural Addresses 19th Century LDA Themes:")
for i, topic in lda_model_1800s.print_topics(-1): 
    print(f"Theme: {i}")
    print(f"Words: {topic}")
    print()

Inaugural Addresses 19th Century LDA Themes:
Theme: 0
Words: 0.006*"protection" + 0.005*"object" + 0.005*"revenue" + 0.005*"extend" + 0.004*"importance" + 0.004*"blessing" + 0.004*"territory" + 0.004*"regard" + 0.004*"opinion" + 0.004*"happy"

Theme: 1
Words: 0.008*"revenue" + 0.007*"business" + 0.006*"legislation" + 0.005*"countryman" + 0.005*"constant" + 0.004*"patriotic" + 0.004*"enterprise" + 0.004*"partisan" + 0.004*"federal" + 0.004*"benefit"

Theme: 2
Words: 0.006*"officer" + 0.006*"revenue" + 0.005*"community" + 0.005*"method" + 0.005*"increase" + 0.005*"expect" + 0.004*"occasion" + 0.004*"debt" + 0.004*"reason" + 0.004*"opinion"

Theme: 3
Words: 0.006*"opinion" + 0.005*"experience" + 0.005*"happiness" + 0.004*"feel" + 0.004*"circumstance" + 0.004*"constitutional" + 0.004*"countryman" + 0.003*"position" + 0.003*"general" + 0.003*"result"

Theme: 4
Words: 0.005*"object" + 0.005*"general" + 0.005*"effect" + 0.005*"exist" + 0.005*"case" + 0.005*"opinion" + 0.005*"grant" + 0.004*"p

In [13]:
print("Inaugural Addresses 20th Century LDA Themes:")
for i, topic in lda_model_1900s.print_topics(-1): 
    print(f"Theme: {i}")
    print(f"Words: {topic}")
    print()

Inaugural Addresses 20th Century LDA Themes:
Theme: 0
Words: 0.007*"island" + 0.007*"rest" + 0.005*"inhabitant" + 0.005*"million" + 0.005*"faithful" + 0.004*"problem" + 0.004*"relation" + 0.004*"treaty" + 0.004*"solve" + 0.004*"establish"

Theme: 1
Words: 0.010*"business" + 0.007*"increase" + 0.006*"ought" + 0.006*"race" + 0.005*"proper" + 0.005*"amendment" + 0.005*"federal" + 0.005*"international" + 0.005*"tariff" + 0.005*"trade"

Theme: 2
Words: 0.008*"task" + 0.006*"civilization" + 0.006*"opportunity" + 0.006*"face" + 0.005*"thought" + 0.005*"industrial" + 0.005*"leadership" + 0.005*"problem" + 0.005*"wish" + 0.005*"today"

Theme: 3
Words: 0.014*"today" + 0.012*"century" + 0.012*"americans" + 0.008*"help" + 0.007*"earth" + 0.007*"democracy" + 0.006*"friend" + 0.006*"child" + 0.006*"begin" + 0.006*"challenge"

Theme: 4
Words: 0.001*"today" + 0.001*"federal" + 0.001*"economic" + 0.001*"business" + 0.001*"opportunity" + 0.001*"increase" + 0.001*"self" + 0.001*"help" + 0.001*"problem" +

In [14]:
print("Inaugural Addresses 21st Century LDA Themes:")
for i, topic in lda_model_2000s.print_topics(-1): 
    print(f"Theme: {i}")
    print(f"Words: {topic}")
    print()

Inaugural Addresses 21st Century LDA Themes:
Theme: 0
Words: 0.001*"thank" + 0.001*"today" + 0.001*"generation" + 0.001*"child" + 0.001*"americans" + 0.001*"like" + 0.001*"woman" + 0.001*"build" + 0.001*"hard" + 0.001*"begin"

Theme: 1
Words: 0.006*"permanent" + 0.006*"excuse" + 0.006*"feel" + 0.005*"tyranny" + 0.005*"deep" + 0.005*"goal" + 0.004*"fulfill" + 0.004*"tradition" + 0.004*"oppression" + 0.004*"idealism"

Theme: 2
Words: 0.018*"thank" + 0.014*"today" + 0.014*"americans" + 0.009*"child" + 0.009*"democracy" + 0.009*"like" + 0.007*"challenge" + 0.007*"promise" + 0.007*"dream" + 0.007*"unity"

Theme: 3
Words: 0.011*"today" + 0.010*"generation" + 0.009*"americans" + 0.007*"woman" + 0.007*"child" + 0.006*"word" + 0.006*"face" + 0.006*"journey" + 0.005*"ideal" + 0.005*"hard"

Theme: 4
Words: 0.001*"today" + 0.001*"child" + 0.001*"generation" + 0.001*"thank" + 0.001*"americans" + 0.001*"like" + 0.001*"word" + 0.001*"forward" + 0.001*"build" + 0.001*"dream"



In [15]:
# 18th century topics visaualization
pyLDAvis.gensim_models.prepare(lda_model_1700s, corpus_1700s, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.093151 -0.024400       1        1  57.301636
3      0.058075 -0.055781       2        1  39.454328
0      0.014773  0.032802       3        1   3.083082
1      0.009269  0.023128       4        1   0.080478
2      0.011033  0.024251       5        1   0.080477, topic_info=             Term      Freq     Total Category  logprob  loglift
378   legislature  3.000000  3.000000  Default  30.0000  30.0000
162         ought  2.000000  2.000000  Default  29.0000  29.0000
247      endeavor  1.000000  1.000000  Default  28.0000  28.0000
248     entertain  1.000000  1.000000  Default  27.0000  27.0000
377     knowledge  2.000000  2.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
38   circumstance  0.000366  1.621701   Topic5  -7.3873  -1.2723
207     sentiment  0.000366  1.714687   Topic5  -7.3876  -1.3284
71   deliberation  0.000365  1.713955   Topic5  -7.3882  -1.3285
52       consider  0.000365  2.367777   Topic5  -7.3890  -1.6525
158      occasion  0.000365  1.872798   Topic5  -7.3891  -1.4180

[308 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5         1  0.553404      adopt
5         2  0.553404      adopt
9         2  1.456416     affair
272       1  0.746031    animate
14        1  0.553707  animosity
...     ...       ...        ...
237       2  0.562578      voice
240       2  1.456719      watch
242       1  0.583793       wise
242       2  0.583793       wise
478       1  1.053897       wish

[151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])

In [16]:
# 19th century topics visaualization
pyLDAvis.gensim_models.prepare(lda_model_1800s, corpus_1800s, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.062147 -0.009161       1        1  32.337869
0      0.021307  0.010396       2        1  26.141746
3      0.069282  0.005800       3        1  17.026551
2     -0.072916 -0.089149       4        1  12.752317
1     -0.079821  0.082114       5        1  11.741517, topic_info=             Term       Freq      Total Category  logprob  loglift
767       revenue  58.000000  58.000000  Default  30.0000  30.0000
1319     business  16.000000  16.000000  Default  29.0000  29.0000
1499       method  15.000000  15.000000  Default  28.0000  28.0000
743       officer  28.000000  28.000000  Default  27.0000  27.0000
564   legislation  24.000000  24.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
432      relation   5.156780  37.154503   Topic5  -5.7869   0.1673
587    protection   5.185442  40.765082   Topic5  -5.7813   0.0801
363      increase   5.166614  40.896172   Topic5  -5.7849   0.0732
351       general   4.433445  55.331588   Topic5  -5.9380  -0.3821
607        safety   4.371441  24.738425   Topic5  -5.9521   0.4088

[392 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1041      1  0.328362  ability
1041      2  0.187635  ability
1041      3  0.093818  ability
1041      4  0.140726  ability
1041      5  0.234544  ability
...     ...       ...      ...
258       3  0.193514  witness
258       4  0.483786  witness
258       5  0.096757  witness
1430      2  0.232962    young
1430      4  0.698886    young

[937 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 3, 2])

In [17]:
# 20th century topics visaualization
pyLDAvis.gensim_models.prepare(lda_model_1900s, corpus_1900s, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.147085 -0.024959       1        1  36.353211
1     -0.108767 -0.080978       2        1  29.565171
2      0.017463 -0.009017       3        1  27.262333
0     -0.049305  0.098874       4        1   6.776824
4     -0.006476  0.016080       5        1   0.042461, topic_info=           Term       Freq      Total Category  logprob  loglift
1584  americans  44.000000  44.000000  Default  30.0000  30.0000
1254    century  45.000000  45.000000  Default  29.0000  29.0000
1539      today  61.000000  61.000000  Default  28.0000  28.0000
1319   business  41.000000  41.000000  Default  27.0000  27.0000
1187    million  22.000000  22.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
573        pass   0.002668  25.209655   Topic5  -7.3106  -1.3892
1584  americans   0.002675  44.221943   Topic5  -7.3082  -1.9488
226        task   0.002671  29.371437   Topic5  -7.3094  -1.5409
1254    century   0.002671  45.753867   Topic5  -7.3096  -1.9843
525       earth   0.002671  31.977818   Topic5  -7.3096  -1.6261

[364 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
259       2  0.536254         abandon
259       4  0.268127         abandon
1306      3  0.911074  accomplishment
918       2  0.099347        activity
918       3  0.794776        activity
...     ...       ...             ...
799       1  0.811001            word
799       2  0.047706            word
799       3  0.143118            word
1430      1  0.789737           young
1430      3  0.157947           young

[548 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1, 5])

In [18]:
# 21st century topics visaualization
pyLDAvis.gensim_models.prepare(lda_model_2000s, corpus_2000s, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.095202  0.069452       1        1  51.161705
2      0.100127 -0.066102       2        1  44.215025
1     -0.084286  0.006280       3        1   4.527665
0     -0.053798 -0.004484       4        1   0.047804
4     -0.057246 -0.005147       5        1   0.047802, topic_info=            Term       Freq      Total Category  logprob  loglift
621        thank  30.000000  30.000000  Default  30.0000  30.0000
1159   democracy  13.000000  13.000000  Default  29.0000  29.0000
673         deep   5.000000   5.000000  Default  28.0000  28.0000
543   generation  19.000000  19.000000  Default  27.0000  27.0000
1590     tyranny   4.000000   4.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
699       family   0.000887   9.261741   Topic5  -7.3618  -1.6073
625   understand   0.000888  10.147315   Topic5  -7.3613  -1.6981
1187     million   0.000886   8.386593   Topic5  -7.3633  -1.5095
525        earth   0.000887  10.159016   Topic5  -7.3617  -1.6997
1597       woman   0.000886  12.639432   Topic5  -7.3632  -1.9196

[338 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1042      1  0.719504       accept
1042      2  0.179876       accept
1397      1  0.756191  achievement
6         1  0.874417      advance
6         2  0.145736      advance
...     ...       ...          ...
1597      1  0.791175        woman
1597      2  0.237352        woman
799       1  0.760145         word
799       2  0.253382         word
636       2  0.929488        write

[242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1, 5])

### Does a certain party have more polarizing speech?


In [19]:
inaugural['party'] = np.array([
    'None',
    'None',
    'Federalist',
    'Democratic-Republican',
    'Democratic-Republican',
    'Democratic-Republican',
    'Democratic-Republican',
    'Democratic-Republican',
    'Democratic-Republican',
    'National Republican',
    'Democratic',
    'Democratic',
    'Democratic',
    'Whig',
    'Democratic',
    'Whig',
    'Democratic',
    'Democratic',
    'Republican',
    'Republican',
    'Republican',
    'Republican',
    'Republican',
    'Republican',
    'Democratic',
    'Republican',
    'Democratic',
    'Republican',
    'Republican',
    'Republican',
    'Republican',
    'Democratic',
    'Democratic',
    'Republican',
    'Republican',
    'Republican',
    'Democratic',
    'Democratic',
    'Democratic',
    'Democratic',
    'Republican',
    'Democratic',
    'Republican',
    'Republican',
    'Republican',
    'Republican',
    'Democratic',
    'Democratic',
    'Republican',
    'Republican',
    'Democratic',
    'Democratic',
    'Republican',
    'Democratic',
    'Republican'
])
inaugural = inaugural[['president_name', 'party', 'president_number', 'date', 'text']]
inaugural.head()

,president_name,party,president_number,date,text
0,George Washington,None,1,1789-04-30 00:00:00+00:00,\nFellow-Citizens of the Senate and of the Hou...
1,George Washington,None,1,1793-03-04 00:00:00+00:00,\nFellow Citizens:\nI AM again called upon by ...
2,John Adams,Federalist,2,1797-03-04 00:00:00+00:00,"\nWHEN it was first perceived, in early times,..."
3,Thomas Jefferson,Democratic-Republican,3,1801-03-04 00:00:00+00:00,\nFriends and Fellow-Citizens:\nCALLED upon to...
4,Thomas Jefferson,Democratic-Republican,3,1805-03-04 00:00:00+00:00,"\nPROCEEDING, fellow-citizens, to that qualifi..."


In [20]:
from afinn import Afinn
import warnings
# ignore groupby warnings:
warnings.filterwarnings("ignore")

afinn = Afinn()

def calculate_sentiment_metrics(tokens):
    """
    Calculates sentiment metrics for tokens
    """
    scores = [afinn.score(token) for token in tokens]
    
    # Remove neutral words for polarization_score
    non_zero_scores = [s for s in scores if s != 0]
    
    return {
        'sentiment_score': sum(scores),
        'positive_words': sum(1 for s in scores if s > 0),
        'negative_words': sum(1 for s in scores if s < 0),
        'sentiment_variance': np.var(non_zero_scores) if non_zero_scores else 0
    }

sentiment_metrics = processed_docs.apply(calculate_sentiment_metrics)
sentiment_df = pd.DataFrame(sentiment_metrics.tolist())

# Combine with original data
inaugural_with_sentiment = pd.concat([inaugural, sentiment_df], axis=1)

def calculate_polarization(group):
    """
    Polarization metrics function for groupby
    """
    return pd.Series({
        'mean_sentiment_variance': group['sentiment_variance'].mean(),
        'total_positive_words': group['positive_words'].sum(),
        'total_negative_words': group['negative_words'].sum(),
        'net_sentiment': group['sentiment_score'].mean(),
        'num_speeches': len(group)
    })

polarization_by_party = inaugural_with_sentiment.groupby('party').apply(calculate_polarization)

polarization_by_party['avg_positive_per_speech'] = (
    polarization_by_party['total_positive_words'] / polarization_by_party['num_speeches']
)
polarization_by_party['avg_negative_per_speech'] = (
    polarization_by_party['total_negative_words'] / polarization_by_party['num_speeches']
)

/Users/calvin/miniconda3/envs/inaugural-address/lib/python3.11/site-packages/afinn/afinn.py:97: DeprecationWarning: invalid escape sequence '\w'
  self._word_pattern = re.compile('\w+', flags=re.UNICODE)


In [21]:
(
    polarization_by_party
    .sort_values('net_sentiment', ascending=False)
    [['total_positive_words', 'total_negative_words', 'net_sentiment', 'num_speeches']]
)

,total_positive_words,total_negative_words,net_sentiment,num_speeches
party,,,,
Whig,478.0,180.0,245.000000,2.0
National Republican,161.0,45.0,227.000000,1.0
Federalist,147.0,40.0,203.000000,1.0
Democratic-Republican,887.0,288.0,194.000000,6.0
Republican,3718.0,1400.0,189.086957,23.0
Democratic,2638.0,1016.0,151.050000,20.0
None,92.0,23.0,69.000000,2.0


In [22]:
(
    polarization_by_party
    .sort_values('avg_positive_per_speech', ascending=False)
    [['avg_positive_per_speech', 'avg_negative_per_speech', 'mean_sentiment_variance']]
)

,avg_positive_per_speech,avg_negative_per_speech,mean_sentiment_variance
party,,,
Whig,239.000000,90.000000,2.588256
Republican,161.652174,60.869565,3.243731
National Republican,161.000000,45.000000,2.751744
Democratic-Republican,147.833333,48.000000,3.147726
Federalist,147.000000,40.000000,3.329578
Democratic,131.900000,50.800000,3.158040
None,46.000000,11.500000,2.705604
